## Exercise II - Acquire and Prep (Wrangle)

### Exercise Intro

- Let's set up an example scenario as perspective for our regression exercises using the Zillow dataset.

- As a Codeup data science graduate, you want to show off your skills to the Zillow data science team in hopes of getting an interview for a position you saw pop up on LinkedIn. You thought it might look impressive to build an end-to-end project in which you use some of their Kaggle data to predict property values using some of their available features; who knows, you might even do some feature engineering to blow them away. Your goal is to predict the values of single unit properties using the obervations from 2017.

- In these exercises, you will complete the first step toward the above goal: acquire and prepare the necessary Zillow data from the zillow database in the Codeup database server.

### 1. Acquire bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, and fips from the zillow database for all 'Single Family Residential' properties.



In [1]:
import pandas as pd
import env as e
import os


def get_zillow():
    # name of cached csv
    filename = "zillow.csv"
    # if cached data exist
    if os.path.isfile(filename):
        df = pd.read_csv(filename)
    # wrangle from sql db if not cached
    else:
        # read sql query into df
        # 261 is single family residential id
        df = pd.read_sql(
            """SELECT yearbuilt,
                       bedroomcnt,
                       bathroomcnt,
                       calculatedfinishedsquarefeet,
                       taxvaluedollarcnt,
                       taxamount,
                       fips
                FROM properties_2017
                WHERE propertylandusetypeid = 261""",
            f"mysql+pymysql://{e.user}:{e.password}@{e.host}/zillow",
        )
        # cache data locally
        df.to_csv(filename, index=False)
    return df

In [2]:
# aquire data
df = get_zillow()

In [3]:
# check out data
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152863 entries, 0 to 2152862
Data columns (total 7 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   yearbuilt                     2143526 non-null  float64
 1   bedroomcnt                    2152852 non-null  float64
 2   bathroomcnt                   2152852 non-null  float64
 3   calculatedfinishedsquarefeet  2144379 non-null  float64
 4   taxvaluedollarcnt             2152370 non-null  float64
 5   taxamount                     2148421 non-null  float64
 6   fips                          2152863 non-null  float64
dtypes: float64(7)
memory usage: 115.0 MB


### 2. Using your acquired Zillow data, walk through the summarization and cleaning steps in your wrangle.ipynb file like we did above. You may handle the missing values however you feel is appropriate and meaningful; remember to document your process and decisions using markdown and code commenting where helpful.



In [4]:
# check the percent of all null values in all columns
df.isnull().mean().sum()

0.010580329542567268

> One percent of the data frame has nulls, we should be able to drop them

In [5]:
# drop all rows with null values
df = df.dropna()

In [6]:
# rename columns for readability

df = df.rename(columns=({"yearbuilt": "year", "bedroomcnt": "bedroom", "bathroomcnt": "bathroom",
            "calculatedfinishedsquarefeet": "square_feet",
            "taxvaluedollarcnt": "property_value",
            "taxamount": "property_tax",
            "fips": "county",
        }
    )
)

In [7]:
# check if nulls are gone
df.isnull().sum()

year              0
bedroom           0
bathroom          0
square_feet       0
property_value    0
property_tax      0
county            0
dtype: int64

In [8]:
df

,year,bedroom,bathroom,square_feet,property_value,property_tax,county
4,2005.0,4.0,2.0,3633.0,296425.0,6941.39,6037.0
6,2011.0,3.0,4.0,1620.0,847770.0,10244.94,6037.0
7,1926.0,3.0,2.0,2077.0,646760.0,7924.68,6037.0
11,1972.0,0.0,0.0,1200.0,5328.0,91.60,6037.0
14,1973.0,0.0,0.0,171.0,6920.0,255.17,6037.0
...,...,...,...,...,...,...,...
2152856,2015.0,4.0,4.0,4375.0,422400.0,13877.56,6037.0
2152858,2015.0,4.0,3.0,2262.0,960756.0,13494.52,6059.0
2152859,2014.0,4.0,4.5,3127.0,536061.0,6244.16,6059.0
2152861,2015.0,3.0,2.5,1974.0,424353.0,5302.70,6059.0


In [9]:
# fips are county codes
# 6037 = Los Angeles County
# 6059 = Orange County
# 6111 = Midland County

# Rename 'fips' to 'county name'
df.county = df.county.map({6037: "LA", 6059: "Orange", 6111: "Ventura"})

In [10]:
# Convert columns to int data type

df = df.astype({"year": int, "bedroom": int, "square_feet": int, "property_value": int})
df

,year,bedroom,bathroom,square_feet,property_value,property_tax,county
4,2005,4,2.0,3633,296425,6941.39,LA
6,2011,3,4.0,1620,847770,10244.94,LA
7,1926,3,2.0,2077,646760,7924.68,LA
11,1972,0,0.0,1200,5328,91.60,LA
14,1973,0,0.0,171,6920,255.17,LA
...,...,...,...,...,...,...,...
2152856,2015,4,4.0,4375,422400,13877.56,LA
2152858,2015,4,3.0,2262,960756,13494.52,Orange
2152859,2014,4,4.5,3127,536061,6244.16,Orange
2152861,2015,3,2.5,1974,424353,5302.70,Orange


### 3. Write a function to split your data into train, validate, and test.


In [7]:
from sklearn.model_selection import train_test_split

def split_data(df):
    """
    This function takes in any DataFrame and a target variable as an argument 
    and returns train, validate, and test dataframes.
    It returns three DataFrames with a printout of their proportion to the original DataFrame.
    """
    train, validate_test = train_test_split(df, train_size=0.6, random_state=123)
    validate, test = train_test_split(validate_test, train_size=0.5, random_state=123)
    print(f"train: {len(train)} ({round(len(train)/len(df), 2)*100}% of {len(df)})")
    print(
        f"validate: {len(validate)} ({round(len(validate)/len(df), 2)*100}% of {len(df)})"
    )
    print(f"test: {len(test)} ({round(len(test)/len(df), 2)*100}% of {len(df)})")

    return train, validate, test


In [12]:
# use function
df = split_data(df)


train: 1284141 (60.0% of 2140235)
validate: 428047 (20.0% of 2140235)
test: 428047 (20.0% of 2140235)


### 4. Store all of the necessary functions to automate your process from acquiring the data to returning a cleaned dataframe with no missing values in your wrangle.py file. Name your final function wrangle_zillow


In [1]:
import os
import pandas as pd
import env as e


def wrangle_zillow():
    """
    This function reads the Zillow data from a cached CSV file if it exists,
    or from a SQL database if it doesn't exist. It then renames the columns
    to more descriptive names.

    Args:
    - None

    Returns:
    - pandas dataframe
    """
    # Name of cached CSV file
    filename = "zillow.csv"
    # If cached data exists, read from CSV file
    if os.path.isfile(filename):
        df = pd.read_csv(filename)
    # Otherwise, read from SQL database
    else:
        df = pd.read_sql(
            """SELECT yearbuilt,
                                   bedroomcnt,
                                   bathroomcnt,
                                   calculatedfinishedsquarefeet,
                                   taxvaluedollarcnt,
                                   taxamount,
                                   fips
                            FROM properties_2017
                            WHERE propertylandusetypeid = 261""",  # 261 is single family residential id
            f"mysql+pymysql://{e.user}:{e.password}@{e.host}/zillow",
        )
        # Cache data locally
        df.to_csv(filename, index=False)
    # Rename columns
    df = df.rename(
        columns={
            "yearbuilt": "year",
            "bedroomcnt": "bedroom",
            "bathroomcnt": "bathroom",
            "calculatedfinishedsquarefeet": "square_feet",
            "taxvaluedollarcnt": "property_value",
            "taxamount": "property_tax",
            "fips": "county",
        }
    )

    # Drop rows with missing values in specific columns 
    df = df.dropna()

    # Map county codes to county names
    df.county = df.county.map({6037: "LA", 6059: "Orange", 6111: "Ventura"})

    # Convert columns to int data type
    df = df.astype({"year": int, "bedroom": int, "square_feet": int, "property_value": int})

    return df

In [2]:
df = wrangle_zillow()
df

,year,bedroom,bathroom,square_feet,property_value,property_tax,county
4,2005,4,2.0,3633,296425,6941.39,LA
6,2011,3,4.0,1620,847770,10244.94,LA
7,1926,3,2.0,2077,646760,7924.68,LA
11,1972,0,0.0,1200,5328,91.60,LA
14,1973,0,0.0,171,6920,255.17,LA
...,...,...,...,...,...,...,...
2152856,2015,4,4.0,4375,422400,13877.56,LA
2152858,2015,4,3.0,2262,960756,13494.52,Orange
2152859,2014,4,4.5,3127,536061,6244.16,Orange
2152861,2015,3,2.5,1974,424353,5302.70,Orange


In [3]:
df.isnull().sum()

year              0
bedroom           0
bathroom          0
square_feet       0
property_value    0
property_tax      0
county            0
dtype: int64

In [4]:
df.describe()

,year,bedroom,bathroom,square_feet,property_value,property_tax
count,2.140235e+06,2.140235e+06,2.140235e+06,2.140235e+06,2.140235e+06,2.140235e+06
mean,1.960968e+03,3.301419e+00,2.240352e+00,1.863194e+03,4.606416e+05,5.616711e+03
std,2.215056e+01,9.325998e-01,9.905489e-01,1.221754e+03,6.771576e+05,7.814563e+03
min,1.801000e+03,0.000000e+00,0.000000e+00,1.000000e+00,2.200000e+01,6.340000e+00
25%,1.949000e+03,3.000000e+00,2.000000e+00,1.258000e+03,1.891660e+05,2.540850e+03
50%,1.958000e+03,3.000000e+00,2.000000e+00,1.623000e+03,3.282960e+05,4.111470e+03
75%,1.976000e+03,4.000000e+00,3.000000e+00,2.208000e+03,5.346060e+05,6.411930e+03
max,2.016000e+03,2.500000e+01,3.200000e+01,9.525760e+05,9.018846e+07,1.078102e+06


In [5]:
df.describe().apply(lambda s: s.apply('{0:.5f}'.format))


,year,bedroom,bathroom,square_feet,property_value,property_tax
count,2140235.00000,2140235.00000,2140235.00000,2140235.00000,2140235.00000,2140235.00000
mean,1960.96754,3.30142,2.24035,1863.19397,460641.62516,5616.71132
std,22.15056,0.93260,0.99055,1221.75416,677157.63568,7814.56280
min,1801.00000,0.00000,0.00000,1.00000,22.00000,6.34000
25%,1949.00000,3.00000,2.00000,1258.00000,189166.00000,2540.85000
50%,1958.00000,3.00000,2.00000,1623.00000,328296.00000,4111.47000
75%,1976.00000,4.00000,3.00000,2208.00000,534606.00000,6411.93000
max,2016.00000,25.00000,32.00000,952576.00000,90188462.00000,1078101.87000


In [9]:
# use function
df = split_data(df)


train: 1284141 (60.0% of 2140235)
validate: 428047 (20.0% of 2140235)
test: 428047 (20.0% of 2140235)
